In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import json
from sklearn.metrics import confusion_matrix as sklearn_cm
from scipy.special import softmax
from scipy.stats import entropy


In [2]:
from collections import Counter


In [3]:
def load_json(directory, filename):
    with open(os.path.join(directory, filename), 'r') as f:
        
        data = json.load(f)
    
    return data


In [4]:
def load_pickle(result_dir, filename):
    with open(os.path.join(result_dir, filename), 'rb') as f:
        data = pickle.load(f)
    
    return data

In [5]:
def calculate_balanced_accuracy(true_labels, predicted_labels, return_type='balanced_accuracy'):
    '''
    used particularly for this 3-classes classification task
    '''
    
    confusion_matrix = sklearn_cm(true_labels, predicted_labels)
    
    class0_recall = confusion_matrix[0,0]/np.sum(confusion_matrix[0])
    class1_recall = confusion_matrix[1,1]/np.sum(confusion_matrix[1])
    class2_recall = confusion_matrix[2,2]/np.sum(confusion_matrix[2])
    
    balanced_accuracy = (1/3)*class0_recall + (1/3)*class1_recall + (1/3)*class2_recall
    
    if return_type == 'all':
        return balanced_accuracy * 100, class0_recall * 100, class1_recall * 100, class2_recall * 100
    elif return_type == 'balanced_accuracy':
        return balanced_accuracy * 100
    else:
        raise NameError('Unsupported return_type in this calculate_balanced_accuracy fn')


### Get test patient study image grouping

In [6]:
StudyOrderInfo_dir = 'YOUR_PATH/ML_DATA/ViewClassifier/seed0/DEV56/study_order_info'

study_order_list = load_json(StudyOrderInfo_dir, 'ForPatientTestSet_val_study_order_list.json')
studylevel_count_dicts = load_json(StudyOrderInfo_dir, 'ForPatientTestSet_val_study_level_count_dicts.json')

study_view_label_list = load_json(StudyOrderInfo_dir, 'ForPatientTestSet_val_study_view_label_list.json')
study_diagnosis_label_list = load_json(StudyOrderInfo_dir, 'ForPatientTestSet_val_study_diagnosis_label_list.json')


In [7]:
Simplified_study_order_list = []
for study in study_order_list:
    if study not in Simplified_study_order_list:
        Simplified_study_order_list.append(study)
        
Simplified_study_order_list

['1176449',
 '2955425',
 '790314',
 '927215',
 '2894038',
 '1976168',
 '2170811',
 '2479348',
 '863452',
 '2213600',
 '1881353',
 '1057994',
 '2780612',
 '3083700']

In [8]:
num_studies = len(Simplified_study_order_list)
print('num_studies: {}'.format(num_studies))

Simplified_study_ImageCount_list = []
Simplified_study_DataIndicesRange_list = []

for study in Simplified_study_order_list:
    print('processing study: {}'.format(study))
    this_study_number_images = 0
    for diagnosis_label, counts in studylevel_count_dicts[study]['diagnosislabels_count'].items():
        print('diagnosis: {}, count: {}'.format(diagnosis_label, counts))
        this_study_number_images += counts
    
    
    assert this_study_number_images == Counter(study_order_list)[study]
        
    Simplified_study_ImageCount_list.append(this_study_number_images)
    print('\n')
    
Simplified_study_DataIndicesEndpoints_list = np.insert(np.cumsum(Simplified_study_ImageCount_list), 0, 0)
print('Simplified_study_DataIndicesEndpoints_list: {}'.format(Simplified_study_DataIndicesEndpoints_list))

for i in range(num_studies):
    Simplified_study_DataIndicesRange_list.append((Simplified_study_DataIndicesEndpoints_list[i], Simplified_study_DataIndicesEndpoints_list[i+1]))
        
print('Simplified_study_DataIndicesRange_list: {}'.format(Simplified_study_DataIndicesRange_list))
    

num_studies: 14
processing study: 1176449
diagnosis: no_AS, count: 70
diagnosis: mild_AS, count: 0
diagnosis: mildtomod_AS, count: 0
diagnosis: moderate_AS, count: 0
diagnosis: severe_AS, count: 0


processing study: 2955425
diagnosis: no_AS, count: 83
diagnosis: mild_AS, count: 0
diagnosis: mildtomod_AS, count: 0
diagnosis: moderate_AS, count: 0
diagnosis: severe_AS, count: 0


processing study: 790314
diagnosis: no_AS, count: 67
diagnosis: mild_AS, count: 0
diagnosis: mildtomod_AS, count: 0
diagnosis: moderate_AS, count: 0
diagnosis: severe_AS, count: 0


processing study: 927215
diagnosis: no_AS, count: 0
diagnosis: mild_AS, count: 0
diagnosis: mildtomod_AS, count: 66
diagnosis: moderate_AS, count: 0
diagnosis: severe_AS, count: 0


processing study: 2894038
diagnosis: no_AS, count: 0
diagnosis: mild_AS, count: 43
diagnosis: mildtomod_AS, count: 0
diagnosis: moderate_AS, count: 0
diagnosis: severe_AS, count: 0


processing study: 1976168
diagnosis: no_AS, count: 0
diagnosis: mild_AS

In [9]:
Counter(study_order_list)

Counter({'1176449': 70,
         '2955425': 83,
         '790314': 67,
         '927215': 66,
         '2894038': 43,
         '1976168': 99,
         '2170811': 64,
         '2479348': 113,
         '863452': 58,
         '2213600': 77,
         '1881353': 61,
         '1057994': 56,
         '2780612': 84,
         '3083700': 94})

### Perform study level integration

### FM + MU

In [10]:
candidate_view_dirs = [
      'PATH_TO_THE_MODEL'
]





candidate_diagnosis_dirs = [
    'PATH_TO_THE_MODEL/ForPatientTestSet_diagnosis_predictions/ema'
]


In [11]:
#View_predictions

In [12]:
#Diagnosis_predictions

In [13]:
confidence_threshold_list = np.linspace(0.8, 0.99, 20)

entropy_threshold_list = np.linspace(0, 1.6, 17)

combination_list = []
prioritizedview_performance_list = []
num_escape_study_list = []
escape_study_list = []

escape_study_viewpredictions_list = []
escape_study_diagnosispredictions_list = []
escape_study_diagnosistruelabels_list = []
escape_study_viewrelevance_list = []
escape_study_viewentropy_list = []

for confidence_threshold in confidence_threshold_list:
    print('########################Current confidence threshold: {}################'.format(confidence_threshold))
    
    for entropy_threshold in entropy_threshold_list:
        combination_list.append((confidence_threshold, entropy_threshold))

        print('Current entropy threshold: {}'.format(entropy_threshold))

        # candidate_view_dirs = [i for i in os.listdir(candidate_view_rootdir) if '.ipynb_checkpoints' not in i]

        for candidate_view in candidate_view_dirs:
        #     candidate_view = os.path.join(candidate_view_rootdir, candidate_view, 'best_predictions_at_ema_val')
            candidate_view = os.path.join(candidate_view, 'ForPatientTestSet_predictions')

#             if not os.path.exists(os.path.join(candidate_view, 'ForPatientTestSet_val_predictions.pkl')):
#                 continue
            View_predictions = load_pickle(candidate_view, 'ForPatientTestSet_val_predictions.pkl')

            for candidate_diagnosis in candidate_diagnosis_dirs:

                Diagnosis_predictions = load_pickle(candidate_diagnosis, 'val_predictions.pkl')

                #perform integration
                Diagnosis_true_labels = Diagnosis_predictions['true_labels']
                View_true_labels = View_predictions['true_labels']

                Diagnosis_predictions_values = Diagnosis_predictions['ema_predictions']
        #         Diagnosis_predictions_values = softmax(Diagnosis_predictions_values, axis=1)

                View_predictions_values = View_predictions['ema_predictions']
                View_predictions_values = softmax(View_predictions_values, axis=1)


                ConfidencedBased_PrioritizedView_predicted_labels = []
                ConfidencedBased_PrioritizedView_predictions = []

                study_true_diagnosis_labels = []
                
                escape_study_count = 0
                escape_studies = []
                
                escape_viewpredictions = []
                escape_diagnosispredictions = []
                escape_diagnosistruelabels = []
                escape_viewrelevance = []
                escape_viewentropy = []

                for idx, study in enumerate(Simplified_study_order_list):
        #             print('Currently aggregating predictions for {}'.format(study).center(100, '-'))
                    this_study_data_indices = list(range(Simplified_study_DataIndicesRange_list[idx][0], Simplified_study_DataIndicesRange_list[idx][1]))

                    this_study_diagnosis_predictions = Diagnosis_predictions_values[this_study_data_indices]
                    this_study_view_predictions = View_predictions_values[this_study_data_indices]
                    this_study_view_true_labels = View_true_labels[this_study_data_indices]

                    this_study_ViewRelevance =  np.sum(this_study_view_predictions[:,:2], axis=1)
                    this_study_ViewEntropy = entropy(this_study_view_predictions, axis=1)

                    #experiment with confidence level
                    confidence_mask = this_study_ViewRelevance >= confidence_threshold
                    
                    entropy_mask = this_study_ViewEntropy <= entropy_threshold
                    
                    final_mask = np.logical_and(confidence_mask, entropy_mask)
                    if np.sum(final_mask)<=0:
                        
                        escape_study_count+=1
                        escape_studies.append(study)
                        
                        this_study_diagnosis_true_labels = Diagnosis_true_labels[this_study_data_indices]
                        assert len(list(set(this_study_diagnosis_true_labels))) == 1, '1 study can only have 1 diagnosis label'
                        this_study_diagnosis_single_label = this_study_diagnosis_true_labels[0]
                        study_true_diagnosis_labels.append(this_study_diagnosis_single_label)

                        escape_viewpredictions.append(this_study_view_predictions)
                        escape_diagnosispredictions.append(this_study_diagnosis_predictions)
                        escape_diagnosistruelabels.append(this_study_diagnosis_single_label)
                        escape_viewrelevance.append(this_study_ViewRelevance)
                        escape_viewentropy.append(this_study_ViewEntropy)
            
            
            #                 print('this_study_diagnosis_predictions: {}'.format(this_study_diagnosis_predictions.shape))
            #                 print('this_study_view_predictions: {}'.format(this_study_view_predictions.shape))

            
                        #Fall back to SimpleAveraging:
                        this_study_SimpleAveraging_prediction = np.mean(this_study_diagnosis_predictions, axis = 0) #a 1x3 vector
            #                 print('this_study_SimpleAveraging_prediction: {}'.format(this_study_SimpleAveraging_prediction))
                        this_study_SimpleAveraging_predicted_label = np.argmax(this_study_SimpleAveraging_prediction)
            #                 print('true_diagnosis:{}, SimpleAverage predicted_diagnosis:{}\n'.format(this_study_diagnosis_single_label, this_study_SimpleAveraging_predicted_label))

                        
                        #record this patient's ConfidenceBased_PrioritizedView predictions and predicted labels
                        ConfidencedBased_PrioritizedView_predictions.append(this_study_SimpleAveraging_prediction)
                        ConfidencedBased_PrioritizedView_predicted_labels.append(this_study_SimpleAveraging_predicted_label)
            #                 print('true_diagnosis:{}, ConfidenceBased_PrioritizedView predicted_diagnosis:{}\n'.format(this_study_diagnosis_single_label, this_study_DiagnosisPredictedLabel_with_ViewRelevance))

                        continue
                    
#                     print('num images above confidence level: {}'.format(np.sum(confidence_mask)))
#                     print('num images below entropy level: {}'.format(np.sum(entropy_mask)))


                    this_study_diagnosis_true_labels = Diagnosis_true_labels[this_study_data_indices]
                    assert len(list(set(this_study_diagnosis_true_labels))) == 1, '1 study can only have 1 diagnosis label'
                    this_study_diagnosis_single_label = this_study_diagnosis_true_labels[0]
                    study_true_diagnosis_labels.append(this_study_diagnosis_single_label)

                
                #     print('this_study_diagnosis_predictions * this_study_ViewRelevance[:, np.newaxis] shape:{}'.format((this_study_diagnosis_predictions * this_study_ViewRelevance[:, np.newaxis]).shape))
                    this_study_DiagnosisPrediction_with_ViewRelevance = this_study_diagnosis_predictions * this_study_ViewRelevance[:, np.newaxis]
                    this_study_DiagnosisPrediction_with_ViewRelevance = this_study_DiagnosisPrediction_with_ViewRelevance[final_mask]
                    this_study_DiagnosisPrediction_with_ViewRelevance = np.mean(this_study_DiagnosisPrediction_with_ViewRelevance, axis=0)
        #                 print('this_study_DiagnosisPrediction_with_ViewRelevance: {}'.format(this_study_DiagnosisPrediction_with_ViewRelevance))


                    this_study_DiagnosisPredictedLabel_with_ViewRelevance = np.argmax(this_study_DiagnosisPrediction_with_ViewRelevance)

                    #record this patient's ConfidenceBased_PrioritizedView predictions and predicted labels
                    ConfidencedBased_PrioritizedView_predictions.append(this_study_DiagnosisPrediction_with_ViewRelevance)
                    ConfidencedBased_PrioritizedView_predicted_labels.append(this_study_DiagnosisPredictedLabel_with_ViewRelevance)
        #                 print('true_diagnosis:{}, ConfidenceBased_PrioritizedView predicted_diagnosis:{}\n'.format(this_study_diagnosis_single_label, this_study_DiagnosisPredictedLabel_with_ViewRelevance))

        
                num_escape_study_list.append(escape_study_count)
                escape_study_list.append(escape_studies)
                
                escape_study_viewpredictions_list.append(escape_viewpredictions)
                escape_study_diagnosispredictions_list.append(escape_diagnosispredictions)
                escape_study_diagnosistruelabels_list.append(escape_diagnosistruelabels)
                escape_study_viewrelevance_list.append(escape_viewrelevance)
                escape_study_viewentropy_list.append(escape_viewentropy)
                
                ConfidencedBased_PrioritizedView_predictions = np.array(ConfidencedBased_PrioritizedView_predictions)

        #                 print('\n\n')
                ConfidenceBased_PrioritizedView_balanced_accuracy = calculate_balanced_accuracy(study_true_diagnosis_labels, ConfidencedBased_PrioritizedView_predicted_labels)
#                 print('view: {}, diagnosis: {}'.format(candidate_view, candidate_diagnosis))
                print('ConfidenceBased_PrioritizedView_balanced_accuracy: {}\n\n'.format(ConfidenceBased_PrioritizedView_balanced_accuracy))
                
                prioritizedview_performance_list.append(ConfidenceBased_PrioritizedView_balanced_accuracy)

                
                
combination_list = np.array(combination_list)
prioritizedview_performance_list = np.array(prioritizedview_performance_list)
num_escape_study_list = np.array(num_escape_study_list)
escape_study_list = np.array(escape_study_list)
escape_study_viewpredictions_list = np.array(escape_study_viewpredictions_list)
escape_study_diagnosispredictions_list = np.array(escape_study_diagnosispredictions_list)
escape_study_diagnosistruelabels_list = np.array(escape_study_diagnosistruelabels_list)
escape_study_viewrelevance_list = np.array(escape_study_viewrelevance_list)
escape_study_viewentropy_list = np.array(escape_study_viewentropy_list)





########################Current confidence threshold: 0.8################
Current entropy threshold: 0.0
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.1
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.2
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.30000000000000004
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.4
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.5
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.6000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.7000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.8
ConfidenceBased_Prioritiz

ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.6000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.7000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.8
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.9
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.0
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.1
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.2000000000000002
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.3
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.4000000000000001
ConfidenceBas

ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


########################Current confidence threshold: 0.91################
Current entropy threshold: 0.0
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.1
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.2
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.30000000000000004
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.4
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.5
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.6000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 0.7000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.6

ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.0
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.1
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.2000000000000002
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.3
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.4000000000000001
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.5
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


Current entropy threshold: 1.6
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664


########################Current confidence threshold: 0.96################
Current entropy threshold: 0.0
ConfidenceBased_PrioritizedView_balanced_accuracy: 41.666666666666664



<ipython-input-13-4834304823b0>:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  escape_study_list = np.array(escape_study_list)
<ipython-input-13-4834304823b0>:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  escape_study_viewpredictions_list = np.array(escape_study_viewpredictions_list)
<ipython-input-13-4834304823b0>:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

In [14]:
np.argmax(prioritizedview_performance_list)


324

In [15]:
##get all the occurance of the max value indices
max_all_indices = np.argwhere(prioritizedview_performance_list == np.max(prioritizedview_performance_list)).flatten().tolist()
max_all_indices


[324]

In [16]:
prioritizedview_performance_list[max_all_indices]

array([52.77777778])

In [17]:
num_escape_study_list[max_all_indices]

array([0])

In [19]:
escape_study_list[max_all_indices]

array([list([])], dtype=object)

In [18]:
combination_list[max_all_indices]

array([[0.99, 0.1 ]])